# Regression tables with `ETable`

Regression tables are generated with `ETable()`. `ETable` inherits from the `MTable` base class, which provides all the core output functionality. This means that `ETable` can generate tables in multiple formats (HTML/GT, docx, LaTeX).

Let us load a sample dataset and run some regressions (here using [PyFixest](https://py-econometrics.github.io/pyfixest/pyfixest.html) but you can also use other packages such as [statsmodels](https://www.statsmodels.org/stable/index.html)):

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import pyfixest as pf
import statsmodels.formula.api as smf
import maketables as mt

# Load sample dataset
df = pd.read_csv("../data/salaries.csv")

# Estimate regressions for examples
est1 = pf.feols("logwage ~ age", data=df)
est2 = pf.feols("logwage ~ age + female", data=df)
est3 = pf.feols("logwage ~ age + female | worker_type", data=df)
est4 = pf.feols("logwage ~ age * female | worker_type + occupation", data=df)



## Basic Usage

The simplest way to create to simply call `ETable()` with a list of models:


In [2]:
mt.ETable([est1, est2, est3, est4])

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a660>

You can also estimate and display multiple regressions with one line of code using the (py)fixest stepwise notation:

In [3]:
mt.ETable(pf.feols("logwage+wage~csw(age, female, age:female)", data=df))

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a900>

## Change order of coefficients

You can change the order in which coefficients are displayed using the `order` argument. This argument takes a list of strings that specify the order of the coefficients. Any coefficients not included in the list will be displayed after the specified ones, in their original order.

In [4]:
mt.ETable([est1, est2, est3, est4], order=["age:female","female", "age"])

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a900>

## Keep and drop variables
For example, we can only keep the variables that we'd like, which keeps all variables that fit the provided regex match.


In [5]:
mt.ETable([est1, est2, est3, est4], keep="age")

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7b770>

We can use the `exact_match` argument to select a specific set of variables:

In [6]:
mt.ETable([est1, est2, est3, est4], keep=["age", "female"], exact_match=True)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a900>

We can also easily **drop** variables via the `drop` argument:

In [7]:
mt.ETable([est1, est2, est3, est4], drop=["age"])

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a900>

## Specify displayed model statistics
If `model_stats` is not specified, ETable automatically selects statistics based on model type. This is handled by model extractors - specialized classes that know how to extract information from different types of regression models (statsmodels, PyFixest, linearmodels, etc.). Each extractor can define which statistics are most relevant for its model type. For example, logit and probit models from statsmodels automatically show Observations, Pseudo R², and Log-likelihood instead of regular R². When you mix different model types in one table (e.g., OLS and probit), ETable displays the union of all relevant defaults, so you see both R² (for OLS) and Pseudo R² + Log-likelihood (for probit). 

You can override this automatic selection by passing a list of strings to `model_stats`. Names of the statistics must match the model's respective attribute names such as "r2", "adj_r2", "N", "r2_within" (see the respective function reference for attributes and omit the leading "_"). The type of standard error estimated can be shown by adding "se_type" to the list. When passing an empty list no model statistics are displayed.

In [8]:
mt.ETable([est1, est2, est3, est4], model_stats=['N','r2','r2_within'])

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7b770>

Model statistics can also be relabeled by passing a dictionary `model_stats_labels`.

In [9]:
mt.ETable([est1, est2, est3, est4], model_stats=['N','r2'], model_stats_labels={"N": "Number of observations", "r2": "R squared"})

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a900>

## Hide fixed effects
We can hide the rows showing the relevant fixed effects by setting `show_fe=False` (for instance when the set of fixed effects is the same for all models and you want to describe this in the text or table notes rather than displaying it in the table).

In [10]:
mt.ETable([est1, est2, est3, est4], model_stats=['r2','N'], show_fe=False)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7b770>

## Formatting coefficients and standard errors
By default coefficients and standard errors are reported. But we can also ask to output p-values or t-values via the `coef_fmt` function argument. This parameter accepts a template string that defines the layout and formatting of each coefficient cell. The `coef_fmt` parameter uses special tokens to represent different elements:

- `b` - The coefficient estimate (beta)
- `se` - The standard error  
- `t` - The t-statistic
- `p` - The p-value
- `ci95l` & `ci95u` - The lower and upper limits of the confidence band
- `\n` - Line break (creates a new line in the cell)
- Literal text - Any other characters (like parentheses, brackets, spaces) appear as-is

For example, `"b* \n (se)"` creates a two-line cell with the coefficient on the first line and the standard error in parentheses on the second line. You can control the format of numbers in two ways: The `digits` parameter provides a simple way to set decimal places for all elements when you're using basic tokens without format specifiers. But you can also add `:format` after a token (e.g., `b:.3f`, `se:.2e`) to specify the exact formatting you want. Here are some common format specifiers:

- `.3f` - 3 decimal places (e.g., 1.234)
- `.2e` - Scientific notation with 2 decimals (e.g., 1.23e-02)  
- `,.0f` - Comma separators, no decimals (e.g., 1,234)
- `,.2f` - Comma separators with 2 decimals (e.g., 1,234.56)
- `:d` - Integer formatting (e.g., 1234)

You can customize whether and where significance stars (\*, \*\*, \*\*\*)  appear by appending `*` after a token in the `coef_fmt` string (Stars are shown based on the `signif_code` thresholds that you can pass as a list to ETable or change in the defaults as explained below.):

- `b*` - Stars after the coefficient (default)
- `t*` - Stars after the t-statistic
- `se*` - Stars after the standard error
- `p*` - Stars after the p-value
- `b:.3f*` - Stars after a formatted coefficient

For example:
- `"b* \n (se)"` - Default behavior, stars appear after coefficient
- `"b \n (t*)"` - Stars appear after the t-statistic instead
- `"b:.3f \n (se:.3f)"` - No stars (default format applied but `*` not specified)

In [11]:
# Show standard errors and p-values
mt.ETable([est1, est2, est3, est4], coef_fmt="b* \n (se) \n [p]", digits=4)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b94a9cad0>

In [12]:
# Example: Show coefficients with 2 and standard errors with 3 decimal places and no stars
mt.ETable([est1, est2, est3], coef_fmt="b:.2f \n (se:.3f)")

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b94a9cad0>

In [13]:
# Scientific notation for small coefficients
mt.ETable([est1, est2, est3], coef_fmt="b:.2e* \n (se:.2e)")

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b94a9cad0>

In [14]:
# Show confidence intervals instead of standard errors & have no stars
mt.ETable([est1, est2, est3], coef_fmt="b \n [ci95l, ci95u]", digits=3)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b94a9cad0>

## Rename variables
You can also rename variables if you want to have a more readable output. Just pass a dictionary to the `labels` argument. Note that interaction terms will also be relabeled using the specified labels for the interacted variables (if you want to manually relabel an interaction term differently, add it to the dictionary).


In [15]:
labels = {
    "logwage": "ln(Wage)",
    "wage": "Wage",
    "age": "Age",
    "female": "Female",
    "tenure": "Years of Tenure",
    "occupation": "Occupation",
    "worker_type": "Worker Type",
    "education": "Education Level"
}

mt.ETable([est1, est2, est3, est4], labels=labels)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a660>

If you want to label the rows indicating the inclusion of fixed effects not with the variable label but with a custom label, you can pass on a separate dictionary to the `felabels` argument.


In [16]:
mt.ETable(
    [est1, est2, est3, est4],
    labels=labels,
    felabels={"occupation": "Occupation Fixed Effects", "worker_type": "Worker Type Fixed Effects"},
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b94a9cad0>

## Setting defaults {#sec-setting-defaults}
You can set default options for `ETable` by changing the class attributes. For example, to change the default model statistics, always use the same labels, digits, and significance levels, you can do the following:

In [17]:
# Set default options for all future ETable calls
mt.ETable.DEFAULT_MODEL_STATS = ["N","r2","r2_within"]
mt.ETable.DEFAULT_COEF_FMT = "b:.3f* \n (se:.3f)"
mt.ETable.DEFAULT_SIGNIF_CODE = [0.001, 0.01, 0.05]

Note that you can also change the defaults for the fixed effects markers:

In [18]:
mt.ETable.DEFAULT_FE_MARKER = ("Yes","No")

You can also set default labels. Importantly, default labels are a class attribute of MTable. The reason is that they then also apply to other table types such as (so far) `DTable` for descriptive statistics.

In [19]:
# Note: Default labels are set as class attribute of MTable 
mt.MTable.DEFAULT_LABELS = labels

In [20]:
mt.ETable([est1, est2, est3, est4])

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b94a9cad0>

## Rename categorical variables

Via the `cat_template` argument, you can rename categorical variables via a specified template *{variable}={value}*. This works when either the variable is categorial in the DataFrame, or the C() or i() operators are used in the regresson formula.
´

In [21]:
est5 = pf.feols("logwage ~ age + female + education", data = df)

mt.ETable([est5],  cat_template = "{variable}: {value}")

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b94a9cad0>

But you can also remove the variable name and only keep the levels (categories) by specifying *cat_template="{value}"*. Note that the labeling of categories also works in interaction terms:

In [22]:
est6 = pf.feols("logwage ~ age + female + education", data = df)
est7 = pf.feols("logwage ~ age + female + education*age", data = df)

mt.ETable([est6, est7],  cat_template="{value}")

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a660>

## Custom model headlines
You can also add custom headers for each model by passing a list of strings to the `model_headers` argument.


In [23]:
mt.ETable(
    [est1, est2, est3, est4],
    model_heads=["OLS", "OLS", "Fixed Effects", "Fixed Effects"],
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a660>

Or change the ordering of headlines having headlines first and then dependent variables using the `head_order` argument. "hd" stands for headlines then dependent variables, "dh" for dependent variables then headlines. Assigning "d" or "h" can be used to only show dependent variables or only headlines. When head_order="" only model numbers are shown.


In [24]:
mod1 = pf.feols("logwage ~ age + female", data=df)
mod2 = pf.feols("wage ~ age + female", data=df)
mod3 = pf.feols("logwage ~ age + female | worker_type + occupation", data=df)
mod4 = pf.feols("wage ~ age + female | worker_type + occupation", data=df)

mt.ETable([mod1, mod2, mod3, mod4], 
          model_heads=["OLS", "OLS", "Fixed Effects", "Fixed Effects"], 
          head_order="hd")
    

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b94a9cad0>

Remove the dependent variables from the headers:

In [25]:
mt.ETable(
    [est1, est2, est3, est4],
    head_order="",
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a660>

## Further custom model information
You can add further custom model statistics/information to the bottom of the table by using the `custom_stats` argument to which you pass a dictionary with the name of the row and lists of values. The length of the lists must be equal to the number of models.


In [26]:
mt.ETable(
    [est1, est2, est3, est4],
    show_fe=False,
    custom_model_stats={
        "Fixed Effects": ["No", "No", "Worker Type", "Worker Type & Occupation"],
        "Data Type": ["Cross-section", "Cross-section", "Cross-section", "Cross-section"],
    },
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b94a9cad0>

## Custom table notes
You can replace the default table notes with your own notes using the `notes` argument.

In [27]:
mynotes = "This table shows wage regressions using a simulated dataset of 1,800 workers. The dependent variable is log wage. Models progressively add controls and fixed effects."
mt.ETable(
    [est1, est2, est3, est4],
    notes=mynotes,
)

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x27b9ae7a660>